In [505]:
using GaussianProcesses, PGFPlots, SumProductNetworks
using StatsFuns, Distributions, JLD, ProgressMeter, MultivariateStats
import SumProductNetworks.add!

### Load the scripts / functions used for SPNs with GP leaves

In [214]:
include("utilFunctions.jl")
include("dataTypes.jl")
include("dataTypeUtilFunctions.jl")
include("computationFunctions.jl")
include("regionGraph.jl")
include("regionGraphUtils.jl")
include("gpUtils.jl")

optimize2! (generic function with 1 method)

## Load the data set

In [521]:
#data = load("../../../../datasets/us_flight_2008/dataset_1M.jld");
X = readcsv("../data/flight/X.csv")
y = readcsv("../data/flight/y.csv");

In [522]:
println(size(X))

(800000, 8)


In [523]:
PGFPlots.Linear(X[:,1], y[:, 1], onlyMarks=true)

! Interruption.
\pgfmathparse@@ ...and \pgfmath@bgroup@stripped @@
                                                  @@\pgfmath@parse@stop 
l.800014 };
         
Error saving as SVG


ERROR (unhandled task failure): InterruptException:
Stacktrace:
 [1] process_events at ./libuv.jl:82 [inlined]
 [2] wait() at ./event.jl:216
 [3] wait(::Condition) at ./event.jl:27
 [4] stream_wait(::Timer, ::Condition, ::Vararg{Condition,N} where N) at ./stream.jl:42
 [5] wait(::Timer) at ./event.jl:357
 [6] (::Base.##300#301{IJulia.#send_stdout,Timer})() at ./event.jl:430


ErrorException: [91mLaTeX error[39m

## Split data set

In [524]:
N = size(X, 1)

800000

In [525]:
Int(ceil(N * 0.8))

640000

In [526]:
Ntrain = 700_000

srand(1234)

ids = collect(1:size(X, 1))
trainingIds = shuffle(ids)[1:Ntrain]
testingIds = shuffle(setdiff(ids, trainingIds))[1:100_000];

In [527]:
Xtest = X[testingIds,:]
ytest = y[testingIds, :]

Xtrain = X[trainingIds, :]
ytrain = y[trainingIds, :];

In [528]:
meanY = mean(ytrain,1)
rmse = sqrt(mean((ytest .- meanY).^2))

println("RMSE (Mean): ", rmse)
println("RMSE (Mean) Gal et al.: 36.61")

RMSE (Mean): 30.965821920780662
RMSE (Mean) Gal et al.: 36.61


In [529]:
a = llsq(Xtrain, ytrain, bias = false)

yhat = Xtest * a
rmse = sqrt(mean((yhat .- ytest).^2))

println("RMSE (LLS): ", rmse)
println("RMSE (LLS) Gal et al.: 34.94")

RMSE (LLS): 29.421745742962788
RMSE (LLS) Gal et al.: 34.94


In [530]:
α = 0.001
a = ridge(Xtrain, ytrain, α, bias = false)

yhat = Xtest * a
rmse = sqrt(mean((yhat .- ytest).^2))

println("RMSE (Ridge): ", rmse)
println("RMSE (Ridge) Gal et al.: 34.94")

RMSE (Ridge): 29.421745742962784
RMSE (Ridge) Gal et al.: 34.94


## create SPN

In [547]:
(N, D) = size(Xtrain)

global gID = 1

numSums = 1
meanFunction = MeanZero();
kernelFunctions = [LinArd(ones(D)*log(5.0)), Mat32Ard(ones(D)*log(5.0), log(1.0)), SE(log(5.0), log(1.0))]
kernelFunctions = [LinIso(log(5.0)), SE(-1., 0.), Mat32Iso(log(5.0), log(1.0))]

kernelPriors = [ [Normal(-2.0,4.0),Normal(-2.0,4.0)] ]

noise = -1.;

# data range
minX = vec(minimum(X, 1)) - 0.1
maxX = vec(maximum(X, 1)) + 0.1

# split size
δ = (maxX - minX) ./ 2

# maximum depth
max_depth = 2
min_samples = 500

overlap = 0.0

srand(1234)

(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos_ND(δ, minX, maxX, max_depth, min_samples, Xtrain);
RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));

println("gpRegions: ", length(gpRegions))

#root = convertToSPN_ND(rootRegion, gpRegions, Xtrain, ytrain, meanFunction, kernelFunctions, noise; overlap = overlap)

#(rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos_ND(δ, minX .+ (δ ./ 2), maxX .+ (δ ./ 2), max_depth);
#RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
#PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));
#root2 = convertToSPN_ND(rootRegion, gpRegions, Xtrain, ytrain, meanFunction, kernelFunctions, noise; overlap = overlap)

Samples: 128
Samples: 456
Samples: 369
Samples: 404
Samples: 306
Samples: 134
Samples: 1


LoadError: [91mAssertionError: [5.6, 15.1, 3.1, 1199.6, 1199.6, 204.1, 1115.1, 20.1] -> [5.6, 15.1, 3.1, 1199.6, 1199.6, 204.1, 1115.1, 20.1][39m

In [545]:
root = convertToSPN_ND(rootRegion, gpRegions, Xtrain, ytrain, meanFunction, 
    kernelFunctions, kernelPriors, noise; overlap = overlap, do_mcmc = false)

LoadError: [91mOutOfMemoryError()[39m

In [461]:
#gpnodes = unique(filter(n -> isa(n, GPLeaf), SumProductNetworks.getOrderedNodes(root_1)));
#map(gnode -> optimize2!(gnode.gp, mean = false, kern = true, noise = true, lik=false), gpnodes);

Base.LinAlg.PosDefException(10)
Base.LinAlg.PosDefException(10)
Base.LinAlg.PosDefException(8)
Base.LinAlg.PosDefException(8)
Base.LinAlg.PosDefException(6)
Base.LinAlg.PosDefException(6)


In [463]:
fill!(root.prior_weights, 1. / length(root))
fill!(root.posterior_weights, 1. / length(root))

spn_update!(root)
spn_posterior(root)

-1351.1404721621793
-1491.7926488871135


In [464]:
yhat = zeros(size(ytest))
    
for (yi, root_) in enumerate([root_1, root_2])
    μ = predict_spn!(root_, Xtest);
    yhat[:,yi] = μ
end

println(sqrt(mean((yhat - ytest).^2)))

1.453069745127591


## Run the code several times...

In [ ]:
Ntrain = 7_000

@showprogress 1 "Running experiments..." for run in 1:20

    ids = collect(1:size(X, 1))
    trainingIds = shuffle(ids)[1:Ntrain]
    testingIds = shuffle(setdiff(ids, trainingIds))[1:100_000];

    Xtest = X[testingIds,:]
    ytest = y[testingIds]

    Xtrain = X[trainingIds, :]
    ytrain = y[trainingIds];

    global gID = 1

    numSums = 1
    meanFunction = MeanZero();
    kernelFunctions = [LinIso(log(5.0)), Mat12Iso(log(5.0), log(1.0)), SE(log(5.0), log(1.0))]

    noise = log(0.5);

    # dimensionsality of data
    D = size(X, 2)

    # split size
    δ = ones(Int, D) * 50

    # data range
    minX = vec(minimum(X, 1)) - 1
    maxX = vec(maximum(X, 1)) + 1

    # maximum depth
    max_depth = 6
    min_samples = 200

    overlap = 5

    (rootRegion, sumRegions, gpRegions, allPartitions) = poonDomingos_ND(δ, minX, maxX, max_depth, min_samples, Xtrain);
    RegionIDs = Dict(r[2] => r[1] for r in enumerate(union(sumRegions, gpRegions)));
    PartitionIDS = Dict(p[2] => p[1] + maximum(values(RegionIDs)) for p in enumerate(allPartitions));
    root = convertToSPN_ND(rootRegion, gpRegions, Xtrain, ytrain, meanFunction, kernelFunctions, noise; overlap = overlap)

    for gpnode in unique(filter(n -> isa(n, GPLeaf), SumProductNetworks.getOrderedNodes(root)))

        if size(gpnode.gp.X, 2) > 0

            try
                optimize2!(gpnode.gp, mean = false, kern = true, noise = true, lik=false)
            catch err
                println("exception: ", err)
                gpnode.gp.logNoise = noise
                for kernel in kernelFunctions
                    if isa(gpnode.gp.k, typeof(kernel))
                            gpnode.gp.k = deepcopy(kernel)
                    end
                end
                optimize2!(gpnode.gp, mean = false, kern = true, noise = false, lik=false)
            end
        end
    end

    fill!(root.prior_weights, 1. / length(root))
    fill!(root.posterior_weights, 1. / length(root))

    spn_update!(root)
    spn_posterior(root)

    test_rmse = spn_rmse(root, Xtest, ytest)

    # save result
    writecsv("flight_$(run)_7K.csv", [test_rmse])
    
end